How to start a webdriver client in node?

In [1]:
var importer = require('../Core');
var webdriverio = require('webdriverio');
var sync = require('wdio-sync')
var fs = require('fs');
var path = require('path');

var TIMEOUT = 10000;
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.json');

var sessions;
var sessionModified;
var getSessions = () => {
    try {
        sessionModified = fs.statSync(SESSIONS_PATH).mtime.getTime();
        sessions = JSON.parse(fs.readFileSync(SESSIONS_PATH)
            .toString());
    } catch (e) {
        sessions = {};
    }

    if(typeof sessions.inactive === 'undefined') {
        sessions.inactive = [];
    }
    if(typeof sessions.active === 'undefined') {
        sessions.active = [];
    }
    return sessions;
};

var updateOrAddSession = (currentSession) => {
    if(fs.existsSync(SESSIONS_PATH)
       && fs.statSync(SESSIONS_PATH).mtime.getTime() > sessionModified) {
        getSessions();
    }
    const updateSession = sessions.active.filter(s => s[1] === currentSession)[0];
    if(typeof updateSession !== 'undefined') {
        if((new Date()).getTime() - updateSession[0] > TIMEOUT / 2) {
            updateSession[0] = (new Date()).getTime();
        } else {
            return;
        }
    } else {
        sessions.active.push([(new Date()).getTime(), currentSession]);
    }
    fs.writeFileSync(
        SESSIONS_PATH,
        JSON.stringify(sessions, null, 4))
}

var getActiveSessions = () => {
    const original = client.requestHandler.sessionID;
    const active = [].concat(sessions.active);
    active.sort((a, b) => b[0] - a[0]);
    return importer.runAllPromises(active.map(session => (resolve) => {
        client.requestHandler.sessionID = session[1];
        client.session()
            .then(s => resolve(s.sessionId))
            .catch(e => resolve(null));
    }))
        .then(available => {
            client.requestHandler.sessionID = original;
            return available
                .filter(sess => typeof sess !== 'undefined' && sess !== null)
                .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
        })
}

var getClientSessions = () => {
    const original = client.requestHandler.sessionID;
    // validate and close each session
    const inactive = sessions.inactive.concat(sessions.active.filter(session => 
            // reuse if lagging longer than 120 seconds?
            (new Date()).getTime() - session[0] > TIMEOUT))
        .filter(session => typeof session[1] !== 'undefined' && session[1] !== null)
    inactive.sort((a, b) => b[0] - a[0]);
    return importer.runAllPromises(inactive.map(session => (resolve) => {
        client.requestHandler.sessionID = session[1];
        client.session()
            .then(s => resolve(s.sessionId))
            .catch(e => resolve(null));
    }))
        .then(available => {
            client.requestHandler.sessionID = original;
            return available
                .filter(sess => typeof sess !== 'undefined' && sess !== null)
                .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
        })
}

var sessionSync = false;
var client;
var createWebdriverClient = (host, port) => {
    if(sessionSync) {
        return new Promise(resolve => setTimeout(() => resolve(), 100))
            .then(() => createWebdriverClient(host, port));
    }
    sessionSync = true;
    const sessions = getSessions();
    
    var webdriverServer = {
        sync: true,
        debug: true,
        host: host || 'localhost',
        port: port || 4444,
        logLevel: 'command',
        baseUrl: 'https://webdriver.io',
        pageLoadStrategy: 'eager',
        connectionRetryTimeout: TIMEOUT,
        desiredCapabilities: {
            browserName: 'chrome',
            chromeOptions: {
                prefs: {
                    'download.default_directory': '/data/downloads',
                    'profile.default_content_setting_values.notifications': 2,
                    'exited_cleanly': true,
                    'exit_type': 'None'
                },
                args: [
                    // TODO: https://superuser.com/questions/461035/disable-google-chrome-session-restore-functionality
                    'user-data-dir=/tmp/profile-' + sessions.active.length,
                    // 'start-fullscreen',
                    'no-sandbox',
                    'disable-session-crashed-bubble',
                    'disable-infobars',
                    'new-window',
                    'disable-geolocation',
                    'disable-notifications',
                    'show-saved-copy',
                    'silent-debugger-extension-api'
                    //'kiosk'
                ]
            }
        },
    };
    
    console.log('Initializing webdriver on ' + webdriverServer.host);
    client = webdriverio.remote(webdriverServer);
    //global.browser = {options: {sync: true}}
    //sync.wrapCommands(client, [], []);
    //global.$ = (...config) => client.element.apply(client, config)
    //global.$$ = (...config) => client.elements.apply(client, config).value
    //global.browser = client;
    client.on('error', function (e) {
        console.log(e);
        this.endAll();
    });
    client.on('end', function () {
        console.log('Daemon: Closing browser');
    });
    client.on('result', function (result) {
        updateOrAddSession(client.requestHandler.sessionID);
        //console.log(result);
    });
    client.addCommand('getClientSessions', getClientSessions);
    client.addCommand('getActiveSessions', getActiveSessions);
    return client
        .getClientSessions()
        // save current session
        .then(validSessions => {
            console.log(validSessions);
            if (validSessions.length == 0) {
                // save new session
                client.requestHandler.sessionID = null;
                // TODO: fix this, doesn't work on second init, keeps opening new windows
                return client.init().session();
            } else {
                // set to first valid
                client.requestHandler.sessionID = validSessions[0];
                return client.session();
            }
        })
        .then(r => {
        console.log(r);
            updateOrAddSession(client.requestHandler.sessionID);
            console.log(client.requestHandler.sessionID);
        })
        /*
        .then(() => client.requestHandler.create({
            path: '/status',
            method: 'GET'
        }))
        */
        .catch(e => console.log(e))
        // Down here at the bottom for safetey
        .then(() => (sessionSync = false))
};
module.exports = createWebdriverClient;
createWebdriverClient;


[Function: createWebdriverClient]

In [2]:
$$.async();var client = createWebdriverClient('localhost', 4444)    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));

Initializing webdriver on localhost
[ 'cd2e4a2685bda6bbc5ee237846e88345' ]


{ sessionId: 'cd2e4a2685bda6bbc5ee237846e88345',
  status: 0,
  value: [ 'CDwindow-1ddf96a5-fd99-4f84-8c7f-a4aa3d6cc388' ] }

In [ ]:
$$.async();client.windowHandles()    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));

How to end the webdriver service?

In [ ]:
client.endAll();


TODO: add decorated logging with screenshots of buttons results can be used: https://github.com/megamindbrian/bots/blob/master/bots/server.js

TODO: transfer state and cache to client

